In [ ]:
import tensorflow as tf
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator  
from keras.applications import densenet  
from keras.models import Sequential, Model, load_model  
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,BatchNormalization, Input
from keras.layers import Activation, Dropout, Flatten, Dense, LeakyReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback  
from keras import regularizers  
from keras import backend as K 
from PIL import Image

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions

import numpy as np
import matplotlib.pyplot as plt

tf.__version__

Using TensorFlow backend.


'1.15.0'

In [ ]:
def rmse(y_pred, y_true):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
def pred_rmse(y_pred, y_true):
  return np.sqrt(np.mean((y_pred.values-y_true.squeeze())**2))

In [ ]:
percentages = pd.read_csv('/content/drive/My Drive/Master/DeepLearning/UNIMIB2016/percentage.csv')

In [ ]:
percentages.describe()

,food_pixels
count,1027.000000
mean,14.155570
std,4.980387
min,1.454018
25%,10.653793
50%,14.272468
75%,17.610183
max,31.496249


In [ ]:
test_index = int(len(percentages)*0.75)
train = percentages.iloc[:test_index,:]
test = percentages.iloc[test_index:,:]

In [ ]:
validation_split = 0.1
train_data_dir = '/content/drive/My Drive/Master/DeepLearning/UNIMIB2016/images'
img_height, img_width = 320, 240
batch_size = 32
nb_epochs = 100

train_datagen = ImageDataGenerator(rescale=1./255,
    validation_split=validation_split, horizontal_flip=True,
     vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0])

test_datagen = ImageDataGenerator(rescale=1./255)  

train_generator = train_datagen.flow_from_dataframe(
     dataframe=train,
    directory=train_data_dir,
    x_col="image_name",
    y_col="food_pixels",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="raw",
    color_mode = "grayscale",
    subset='training')

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory=train_data_dir,
    x_col="image_name",
    y_col="food_pixels",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="raw",
    color_mode = "grayscale",
    subset='validation')

test_generator = test_datagen.flow_from_dataframe(
     dataframe=test,
    directory=train_data_dir,
    x_col="image_name",
    y_col="food_pixels",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode = "grayscale",
    class_mode="raw", shuffle=False)

Found 693 validated image filenames.
Found 77 validated image filenames.
Found 257 validated image filenames.


In [ ]:
inputs = Input(shape=(img_height, img_width, 1))

x = Conv2D(16, (3, 3), padding='same')(inputs)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Conv2D(16, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.5)(x)

x = Conv2D(8, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Conv2D(8, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.5)(x)

x = Flatten()(x)
x = Dense(16)(x)
x = LeakyReLU()(x)

x = Dense(4)(x)
x = LeakyReLU()(x)

x = Dense(1, activation="linear")(x)
model = Model(inputs, x)

In [ ]:
#opt = Adam(lr=1e-3, decay=1e-3 / 200)
es = EarlyStopping(monitor='val_loss', patience=10, mode='auto', restore_best_weights=True)
r_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, mode='auto', min_lr=1e-6)
model.compile(loss="mean_squared_error", optimizer="adam", metrics=['mae', 'mse', rmse])

In [ ]:
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 320, 240, 1)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 320, 240, 16)      160       
_________________________________________________________________
batch_normalization_21 (Batc (None, 320, 240, 16)      64        
_________________________________________________________________
leaky_re_lu_35 (LeakyReLU)   (None, 320, 240, 16)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 320, 240, 16)      2320      
_________________________________________________________________
batch_normalization_22 (Batc (None, 320, 240, 16)      64        
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 320, 240, 16)      0   

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs, callbacks = [es, r_lr])

Epoch 1/100
21/21 [==============================] - 13s 596ms/step - loss: 170.2396 - mean_absolute_error: 9.6178 - mean_squared_error: 170.2396 - rmse: 10.8920 - val_loss: 118.9055 - val_mean_absolute_error: 9.1452 - val_mean_squared_error: 118.9055 - val_rmse: 10.8826
Epoch 2/100
21/21 [==============================] - 11s 520ms/step - loss: 34.2876 - mean_absolute_error: 4.6987 - mean_squared_error: 34.2876 - rmse: 5.7441 - val_loss: 110.4989 - val_mean_absolute_error: 9.6674 - val_mean_squared_error: 110.4989 - val_rmse: 10.4872
Epoch 3/100
21/21 [==============================] - 10s 490ms/step - loss: 21.8779 - mean_absolute_error: 3.7735 - mean_squared_error: 21.8779 - rmse: 4.6445 - val_loss: 118.9287 - val_mean_absolute_error: 9.4943 - val_mean_squared_error: 118.9287 - val_rmse: 10.8738
Epoch 4/100
21/21 [==============================] - 11s 530ms/step - loss: 16.9824 - mean_absolute_error: 3.3064 - mean_squared_error: 16.9824 - rmse: 4.0890 - val_loss: 71.8993 - val_mean_

In [ ]:
import numpy as np
print("RMSE: "+str(pred_rmse(test["food_pixels"], model.predict(test_generator))))

RMSE: 4.567784853379181


In [ ]:
model.save('/content/drive/My Drive/Master/DeepLearning/UNIMIB2016/models/raw_model.h5')